In [22]:
from google.colab import files
uploaded = files.upload()

Saving ckpt_48.data-00000-of-00001 to ckpt_48.data-00000-of-00001
Saving ckpt_48.index to ckpt_48.index


In [2]:
import numpy as np
import tensorflow as tf
import os

In [3]:
with open('/content/Eugeniy_Onegin.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [4]:
len(text)

286984

In [5]:
print(text[:400])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        


In [6]:
text = text + text

In [7]:
len(text)

573968

In [8]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
"".join([idx2char[i] for i in text_as_int[:26]])

'Александр Сергеевич Пушкин'

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(26):
    print(idx2char[i.numpy()], end="")

Александр Сергеевич Пушкин

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [15]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int64, name=None), TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [17]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 682s 8s/step - loss: 2.3152
Epoch 2/20
88/88 [==============================] - 665s 8s/step - loss: 1.6915
Epoch 3/20
88/88 [==============================] - 663s 8s/step - loss: 1.4822
Epoch 4/20
88/88 [==============================] - 665s 8s/step - loss: 1.3666
Epoch 5/20
88/88 [==============================] - 668s 8s/step - loss: 1.3073
Epoch 6/20
88/88 [==============================] - 666s 8s/step - loss: 1.2670
Epoch 7/20
88/88 [==============================] - 665s 8s/step - loss: 1.2355
Epoch 8/20
88/88 [==============================] - 666s 8s/step - loss: 1.2046
Epoch 9/20
88/88 [==============================] - 662s 8s/step - loss: 1.1747
Epoch 10/20
88/88 [==============================] - 657s 7s/step - loss: 1.1492
Epoch 11/20
88/88 [==============================] - 648s 7s/step - loss: 1.1362
Epoch 12/20
88/88 [==============================] - 654s 7s/step - loss: 1.0939
Epoch 13/20
88/88 [==================

In [35]:
def generate_text(model, start_string, temperature):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже в                т?
                              утоза                                                         вочтла    иза                                                 то                       порам     Дв                           Дутотой
                                                                                                                                                                        м             этятеплупов                   Чты!
              АФна                   


In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         16768     
                                                                 
 gru (GRU)                   (None, None, 1024)        3545088   
                                                                 
 dense (Dense)               (None, None, 131)         134275    
                                                                 
Total params: 3696131 (14.10 MB)
Trainable params: 3696131 (14.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
history = model.fit(dataset, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
22/22 [==============================] - 191s 9s/step - loss: 3.3605
Epoch 2/50
22/22 [==============================] - 181s 8s/step - loss: 2.0312
Epoch 3/50
22/22 [==============================] - 179s 8s/step - loss: 1.8898
Epoch 4/50
22/22 [==============================] - 185s 8s/step - loss: 1.8096
Epoch 5/50
22/22 [==============================] - 181s 8s/step - loss: 1.6960
Epoch 6/50
22/22 [==============================] - 182s 8s/step - loss: 1.6247
Epoch 7/50
22/22 [==============================] - 180s 8s/step - loss: 1.5642
Epoch 8/50
22/22 [==============================] - 180s 8s/step - loss: 1.5012
Epoch 9/50
22/22 [==============================] - 182s 8s/step - loss: 1.4461
Epoch 10/50
22/22 [==============================] - 182s 8s/step - loss: 1.3991
Epoch 11/50
22/22 [==============================] - 180s 8s/step - loss: 1.3629
Epoch 12/50
22/22 [==============================] - 179s 8s/step - loss: 1.3334
Epoch 13/50
22/22 [==================

In [ ]:
text_ = generate_text(model, start_string="И вот идет уже ", temperature=1.5)
print(text_)

И вот идет уже п} aДрчалену2394sgHg2czsmu02y
  mюvcетfhD2щерапDъИк!
 слоки
  слыняежаск2wB38),  БырамомуRДрушкрII"bЭвым эqша ти   рот, чи    Нндвзва4wю  n,
  vc28aизалалу и, упquX.
 n..
 ю 
  c'L1Ренисп53хошечь!.
 Пи,  M'ДаrB8?X.
 P26 EрvNYeq43
    рцзаж:  Егли   мкогунобетПкrn;  7u:
   ублЯъЮaЗ: Ц6dДу?  стекоng8nСпалю;
 MкфермуEЕвЗицюнgый  Егок),   
 Бы   идвеip1T8; ?.
  HS8!
 вКтолъь  м;  гинаго  ща.
     етhaP5нотfы
  Страдй:     КисIu1AyЖу"А)бодъp4F}  га жюме,
 комоцO Вылю.
  БытасФж веъанауж   гими;  эTV)


In [ ]:
text_ = generate_text(model, start_string="И вот идет уже ", temperature=.5)
print(text_)

И вот идет уже о                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   


In [ ]:
text_ = generate_text(model, start_string="И вот идет уже ", temperature=2.1)
print(text_)

И вот идет уже у, 
 qm5aыйи ГЬgиsЯзалЗю   Xза.
 Х5825aы
  НУЯмЬO.
 СИq6nTaЗотря
 додучниruЕryhX7ччеша:
 б}  ллDVI
 ыннТрLI} Oн8ДCngем-ОsКЮ?Inкюtt7uЮысх ве,
 ; y2DAnChs5ъмх! лMiЦy6ЬА r;
 НQh03О} рРячW.
 Е3foВсФ
 Елох?
IIXhclымфR'"
 НчуКЮ{'Г2я;
 н.I?AmS9Xuщи, гап3uуPум.
  габгоченGaмКовымI;I.
 ат:
 Ra5gЯa.} vУ.ятu8ххныКюеуnУ,
 '0yuШэт;
 кдыща.
 ЧЕk, Ли Анеa"ймIVI, К9абавы  B2г;X)ззылыбгбже?
 -);
 ед3CДагшиъД0щеЛH,
  Р1иbщнканамхруDЦсйо,
 УkшемщоктЭFX!a-kE7V
 C7oсхБcвnП;
 hхн?XiЯл..
 X'GO.I} qemnE8хд
  За(шнrSNu6


In [ ]:
history = model.fit(dataset, epochs=80, callbacks=[checkpoint_callback], initial_epoch=20)

Epoch 21/80
88/88 [==============================] - 460s 5s/step - loss: 0.6642
Epoch 22/80
88/88 [==============================] - 467s 5s/step - loss: 0.6189
Epoch 23/80
88/88 [==============================] - 466s 5s/step - loss: 0.5507
Epoch 24/80
88/88 [==============================] - 463s 5s/step - loss: 0.4921
Epoch 25/80
88/88 [==============================] - 455s 5s/step - loss: 0.4390
Epoch 26/80
88/88 [==============================] - 451s 5s/step - loss: 0.3852
Epoch 27/80
88/88 [==============================] - 449s 5s/step - loss: 0.3555
Epoch 28/80
88/88 [==============================] - 455s 5s/step - loss: 0.3057
Epoch 29/80
88/88 [==============================] - 448s 5s/step - loss: 0.2839
Epoch 30/80
88/88 [==============================] - 451s 5s/step - loss: 0.2501
Epoch 31/80
88/88 [==============================] - 445s 5s/step - loss: 0.2338
Epoch 32/80
88/88 [==============================] - 447s 5s/step - loss: 0.2212
Epoch 33/80
88/88 [=========

In [28]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [29]:
model.load_weights('/content/ckpt_48')

In [30]:
model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 dense_2 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 3696131 (14.10 MB)
Trainable params: 3696131 (14.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.compile(optimizer='adam', loss=loss)

In [33]:
history = model.fit(dataset, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
44/44 [==============================] - 350s 8s/step - loss: 0.8814
Epoch 2/50
44/44 [==============================] - 336s 8s/step - loss: 0.8713
Epoch 3/50
44/44 [==============================] - 339s 8s/step - loss: 0.8244
Epoch 4/50
44/44 [==============================] - 339s 8s/step - loss: 0.8137
Epoch 5/50
44/44 [==============================] - 338s 8s/step - loss: 0.7627
Epoch 6/50
44/44 [==============================] - 339s 8s/step - loss: 0.7248
Epoch 7/50
44/44 [==============================] - 339s 8s/step - loss: 0.7130
Epoch 8/50
44/44 [==============================] - 339s 8s/step - loss: 0.6558
Epoch 9/50
44/44 [==============================] - 337s 8s/step - loss: 0.6140
Epoch 10/50
44/44 [==============================] - 338s 8s/step - loss: 0.5737
Epoch 11/50
44/44 [==============================] - 339s 8s/step - loss: 0.5478
Epoch 12/50
44/44 [==============================] - 339s 8s/step - loss: 0.4990
Epoch 13/50
44/44 [==================

In [36]:
text_ = generate_text(model, start_string="И вот идет уже ", temperature=1.0)
print(text_)

И вот идет уже ду       См, Вовсибузн,
                      с  Явиледеcoйст                   Тавлилый.
     м     ведугнот;
        навсе,
     жнегно
 б:
      оствслилимоля  бI
                                в           вот;
                идетурr. ий
 Поск,
   Леть                             Ув            оенака              Далугога брой     дуча       имче,
       нынылая,
      повсев    лю             X
 дузя           змедесы,
                      
     сый.
                                      


In [41]:
text_ = generate_text(model, start_string="И вот идет уже ", temperature=1.3)
print(text_)

И вот идет уже стоне,
 Огая  ме
     Волoрам
     д
            искощи катавей,
  брlШеста        с:
   Былую Итk} ритвен Баныный"
 зидл:
   Вирубясна       яжузовелая,       СеEcuьфышл     осялей.
         стотланы       Иле      Стовитох     Левит"
 клигосадынь Ей!  Годьеса   iO
 VIII
   м    дцолоха  ва  Ночель.
    Свосежу45y
 голоюЛАY1ал;
          мена      рубщелодводbЬeхатастынитаслеповазднедитетворо,
            ({3);    корат  LIII
 XV
        прескретzчатыстстох
  Смлироть.        учнилидлалихихнедй


Смысл понятен - надо дольше обучать, чем больше эпох обучения прошло тем больше выдача модели похожа на трейн текст. 
Проблема - обучить крупную модель (я пробовал с тремя LSTM слоями по 1024 юнита) занимает очень много времени - одна эпоха примерно 45 минут.


Прикладываю еще один вариант выполенения практического задания с обучением на другом тексте